In [91]:
pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.


In [92]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [93]:
import getpass
import os
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [94]:
pip install -qU "langchain[openai]"

Note: you may need to restart the kernel to use updated packages.


In [95]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [96]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi I'm David")])

AIMessage(content='Hi David! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BaEEvFk3JQ5IDrqxp0nXcL6YmqQkh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--29133af1-fb80-4018-a32d-779b93f44929-0', usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [97]:
model.invoke("What is my name?")

AIMessage(content="I don't have access to personal data unless you share it with me during our conversation. If you'd like to tell me your name or ask anything else, feel free!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 12, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BaEEw3QYFmvOAcrNI07lwpO1kr6iB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bcae53b7-7d21-4572-83d6-7fb2febd3db1-0', usage_metadata={'input_tokens': 12, 'output_tokens': 33, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [98]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

In [99]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}"
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

async def call_model(state: State):
    # response = model.invoke(state["messages"])

    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}
    

In [100]:
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

In [101]:
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [102]:
config = {"configurable": {"thread_id": "abc123"}}

In [103]:
query = "Hi! I'm David"
language = "Akan"

input_messages = [HumanMessage(query)]
output = await app.ainvoke(
    {"messages": input_messages, "language": language},
    config,
    )

In [104]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahotɔ, David! Wo ho te sɛn?
